In [1]:
from sqlalchemy import create_engine
import pandas as pd
from fuzzywuzzy import process, fuzz
import geopy
from geopy.geocoders import Nominatim
from os import environ
from googlemaps import Client
import math

In [2]:
connection_url =  environ["ICEURL"]

In [3]:
dataframe = pd.read_sql_table("locations", connection_url, "public")

In [4]:
trac_locations = pd.read_csv('trac_locations_corrected.csv')

In [5]:
choices = trac_locations['Facility Name']
j = 0
k = 0
geolocator = Nominatim(user_agent="iceicedata")
geopy.geocoders.options.default_timeout = 7
client = Client(environ["APIKEY"])
for i in range(0, len(dataframe)):
    if math.isnan(dataframe.iloc[i]['lat']):
        state = ''
        city = ''
        lat = math.nan
        lng = math.nan
        if dataframe.iloc[i]['state'] is not None:
            state = dataframe.iloc[i]['state']
            if dataframe.iloc[i]['city'] is not None:
                city = dataframe.iloc[i]['city']
            else:
                city = ''
        else:
            name = dataframe.iloc[i]['name']
            match = process.extractOne(name, choices, scorer=fuzz.token_set_ratio)
            if match[1]>92:
                city = trac_locations.at[match[2], 'City']
                state = trac_locations.at[match[2], 'State']
                location = geolocator.geocode(city+', '+state)
                if location is not None:
                    lng = location.longitude
                    lat = location.latitude
                else:
                    lng = math.nan
                    lat = math.nan

        if state == '':
            # Try google
            place = client.places(name)
            if place['status'] == 'OK':
                lat = place['results'][0]['geometry']['location']['lat']
                lng = place['results'][0]['geometry']['location']['lng']
                location = geolocator.reverse(str(lat)+', '+str(lng))
                if location is not None:
                    if 'city' in location.raw['address']:
                        city = location.raw['address']['city']
                    elif 'hamlet' in location.raw['address']:
                        city = location.raw['address']['hamlet']
                    elif 'town' in location.raw['address']:
                        city = location.raw['address']['town']
                    elif 'village' in location.raw['address']:
                        city = location.raw['address']['village']
                    elif 'locality' in location.raw['address']:
                        city = location.raw['address']['locality']
                    else:
                        city = ''
            else:
                k += 1
        if not math.isnan(lat):
            dataframe.at[i,'city'] = city
            dataframe.at[i,'state'] = state
            dataframe.at[i,'lon'] = lng
            dataframe.at[i, 'lat'] = lat
            j += 1
        else:
            print('no location for ', city, state)
            k += 1          
    else:
        j += 1
print(j,' matches and ', k, ' fails.')

no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for  New York NY
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no location for   
no

### Notes
Misspellings:  In the windows above messages "<city>, <state> has no location" probably indicates a misspelling in the trac data.



In [6]:
dataframe.to_csv('ice_locations.csv')